# Задание к занятию «Рекомендации на основе скрытых факторов»

Описание задания:
Что делать
1. Установить implicit;
2. Взять датасет last.fm (урезанный или полный);
3. Разбить датасет на обучающую и тестовую выборки;
4. Построить на обучающей выборке хотя бы две модели из пакета implicit:
    − kNN по косинусной мере
    − ALS
5. Получить рекомендации на тестовой выборке для обученных моделей;
6. Сравнить метрики качества обученных моделей на тестовой выборке с помощью mrec (или иным способом).

In [28]:
import pandas as pd
import numpy as np

In [29]:
# plots snippet
from matplotlib import pyplot as plt
import seaborn as sns
from pylab import rcParams

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
%config InlineBackend.figure_format = 'png'
rcParams['figure.figsize'] = (16, 6)

In [30]:
! ls

lastfm.als.pred.tsv  lastfm.test.tsv      latent-factors.ipynb
lastfm.knn.pred.tsv  lastfm_small.tsv


In [31]:
column_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv('lastfm_small.tsv', sep='\t', header=None, names=column_names)
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


## Подготовка данных

In [32]:
data.fillna("None", inplace=True)
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
data["plays"] = data["total-plays"].astype(np.double)
data.drop(["artist-name", "artist-mbid", "user", "total-plays"], axis=1, inplace=True)
data.head()

,user_id,artist_id,plays
0,1,15531,2137.0
1,2,63469,1099.0
2,2,46858,897.0
3,2,15968,717.0
4,2,48969,706.0


In [33]:
data.describe()

,user_id,artist_id,plays
count,1000000.000000,1000000.000000,1000000.000000
mean,10233.925995,33678.492236,216.606950
std,5912.022449,19230.330182,604.378024
min,1.000000,1.000000,1.000000
25%,5119.000000,17298.000000,34.000000
50%,10238.000000,34544.000000,94.000000
75%,15348.000000,49488.000000,225.000000
max,20466.000000,66799.000000,135392.000000


In [34]:
test_indices = np.random.choice(data.index.values, replace=False, size=int(len(data.index.values) * 0.2))
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)
train_data.shape, test_data.shape

((800000, 3), (200000, 3))

In [35]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 3
нет в тестовой выборке, но есть в обучающей: 22
всего пользователей: 20466


In [36]:
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
test_data.drop(test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index, inplace=True)
train_data.drop(train_data[train_data["user_id"].isin(user_ids_to_exclude).values].index, inplace=True)

/Users/a.burtsev/miniconda3/envs/netology/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
train_data.shape, test_data.shape

((799789, 3), (199997, 3))

In [38]:
test_data.to_csv('lastfm.test.tsv', sep='\t', index=False, header=False)

In [39]:
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix

In [40]:
# функция, которая красиво печатает информацию о разреженных матрицах

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [41]:
plays = coo_matrix((
    train_data["plays"].astype(np.float),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66800, 20467)
Ненулевых элементов в матрице: 799776
Доля ненулевых элементов: 0.0005849755514134189
Среднее значение ненулевых элементов: 216.31050319089346
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


## Обучение и предсказания

### ALS

In [42]:
from implicit.als import AlternatingLeastSquares
from multiprocessing import cpu_count

In [43]:
model = AlternatingLeastSquares(factors=100, num_threads=cpu_count())
model.fit(plays)

In [44]:
user_plays = plays.T.tocsr()

predict_artist = []

for user_id in test_data['user_id'].unique():
    for artist_id, score in model.recommend(user_id, user_plays):        
        predict_artist.append((user_id, artist_id, score))

als_predictions = pd.DataFrame(predict_artist, columns=['user_id', 'artist_id', 'plays'])
als_predictions.to_csv('lastfm.als.pred.tsv', sep='\t', index=False, header=False)

### KNN

In [45]:
from implicit.nearest_neighbours import CosineRecommender

In [46]:
model = CosineRecommender()
model.fit(plays)
sparse_info(model.similarity)

Размерности матрицы: (66800, 66800)
Ненулевых элементов в матрице: 1215461
Доля ненулевых элементов: 0.0002723880831152067
Среднее значение ненулевых элементов: 0.44390724490716377
Максимальное значение ненулевых элементов: 1.0000000000000078
Минимальное значение ненулевых элементов: 0.0


In [47]:
user_plays = plays.T.tocsr()

predict_artist = []

for user_id in test_data['user_id'].unique():
    for artist_id, score in model.recommend(user_id, user_plays):        
        predict_artist.append((user_id, artist_id, score))

knn_predictions = pd.DataFrame(predict_artist, columns=['user_id', 'artist_id', 'plays'])
knn_predictions.to_csv('lastfm.knn.pred.tsv', sep='\t', index=False, header=False)

## Метрики качества

In [48]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [49]:
als_test_predictions = pd.merge(test_data, als_predictions, on=['user_id', 'artist_id'], suffixes=('_test', '_pred'))

In [50]:
knn_test_predictions = pd.merge(test_data, knn_predictions, on=['user_id', 'artist_id'], suffixes=('_test', '_pred'))

### RMSE

In [51]:
print('ALS', sqrt(mean_squared_error(als_test_predictions['plays_test'], als_test_predictions['plays_pred'])))
print('KNN', sqrt(mean_squared_error(knn_test_predictions['plays_test'], knn_test_predictions['plays_pred'])))

ALS 969.9084665130302
KNN 1419.5128903400305


### MREC

In [52]:
from collections import defaultdict
from mrec import load_sparse_matrix
from mrec.evaluation.metrics import compute_main_metrics, compute_hit_rate
from mrec.evaluation import Evaluator
from mrec.evaluation.metrics import print_report

evaluator = Evaluator(compute_main_metrics, max_items=20)
testdata = load_sparse_matrix('tsv', 'lastfm.test.tsv').tocsr()

#### ALS

In [53]:
all_metrics = defaultdict(list)

cum_metrics, count = evaluator.process(testdata, 'lastfm.als.pred.tsv', 0, testdata.shape[0])
if cum_metrics is not None:
    for m in cum_metrics:
        all_metrics[m].append(float(cum_metrics[m]) / count)
        
print_report(['ALS'], [all_metrics])

ALS
mrr            0.3132 +/- 0.0000
prec@5         0.1377 +/- 0.0000
prec@10        0.1128 +/- 0.0000
prec@15        0.0752 +/- 0.0000
prec@20        0.0564 +/- 0.0000


#### KNN

In [54]:
all_metrics = defaultdict(list)

cum_metrics, count = evaluator.process(testdata, 'lastfm.knn.pred.tsv', 0, testdata.shape[0])
if cum_metrics is not None:
    for m in cum_metrics:
        all_metrics[m].append(float(cum_metrics[m]) / count)
        
print_report(['KNN'], [all_metrics])

KNN
mrr            0.0483 +/- 0.0000
prec@5         0.0164 +/- 0.0000
prec@10        0.0147 +/- 0.0000
prec@15        0.0098 +/- 0.0000
prec@20        0.0074 +/- 0.0000
